In [3]:
import os
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error, median_absolute_error

import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Importing Dataset

In [5]:
# Importing processed dataframe
path_project = "C:/Users/Conno/Documents/Career/Projects/Hospital_Charges/tree_based_models"

os.chdir(path_project)
plots_dir = 'xgb_plots' # stores plots in plot folder

df = pd.read_csv("../df_processed.csv")

# do this later in data_cleaning file
bool_cols = df.select_dtypes(include=['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)

#df.dtypes

## Train-Test Split

In [6]:
X = df.drop(columns = ["charges"])
y = df["charges"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 32)

# XGBoost

XGBoost stands for "eXtreme Gradient Boosting" and it is a specific implementation of the gradient boosting method. The two most important changes are:

1. computing second order derivatives
    1. Gradient boosting uses the loss function of the base model (e.g decision tree) as a proxy for minimizing the error of the overall model, XGBoost uses the 2nd order derivative as an approximation.
    2. Uses L1 and L2 regularization to improve model generalization.
    
XGBoost has become an industry standard in the data science community and, to put it crudely, is the silver bullet in the data science toolkit. This is because training is very fast, can be distributed across compute clusters, and has achieved state of the art perfromance in several competitions. The beauty behind it is not just its accuracy but its scalability.

XGBoost also handles sparse data in an effective way (`NA` data) thus we will be using testing the algorithm with the processed dataset AND the full, unclean dataset.

## Mathematics

In XGBoost, the objective function is:

$$\mathcal{L}(\phi) = \sum_{i}l(\hat{y}_i, y_i) + \sum_{k}\Omega(f_k)$$

$$\Omega(f) = \gamma T + \frac{1}{2}\lambda \lVert w \rVert^2$$

Where $l$ is a differentiable convex loss function measuring the difference between the true and predicted values. XGBoost creates trees by optimizing (minimizing) this objective as much as possible.

The second term $\Omega$ is the regularization term for the k-th tree (penalizes the complexity of the model). The regularization term helps smooth the final learnt weights to avoid overfitting. 
- $T$ is the number of terminal nodes in the tree $f$. 
- $\gamma$ is the regularization parameter that penalizes the number of leaves in a tree (prunes the tree). 
- $w$ represents the vector of leaf weights (the "output" or "prediction" values at the leaves)
- $\lVert w \rVert^2$ is the sum of squared weights. 
- $\lambda$ is a regularization parameter that penalizes large weights, encouraging the model to keep the leaf values smaller and thereby reducing the risk of overfitting. For XGBoost, it seems that $\lambda$ refers to L2 regularization and $\alpha$ would refer to L1 regularization. 

XGBoost's more sophisticated regularization framework allows for better control over the complexity of the model, which leads to improved generalization and reduced risk of overfitting.

Traditional optimization methods, such as gradient descent, operate in Euclidean space (x, y, z coordinates, etc). These methods typically involve optimizing a funtion with repsect to a set of parameters, where the parameters are represented as points in Euclidean space. XGBoost includes not only parameters in Euclidean space but also functions, specifically decision trees. Decision trees are more complex structures that cannot be eaily optimized using tradional methods designed for Euclidean space. This is because the tree structure as parameters (optimizing over trees involves discrete choices, not a continious process moving in Euclidean space). Also because the process of growing trees involves non-differentiable operations such as choosing the best split based on some criterion (e.g., information gain, Gini impurity). 

Instead of optimizing over the entire tree structure at once, XGBoost uses an additive approach where a model is built incrementally, like normal gradient boosting. 

### Algorithm

1. Start with an initial prediction, e.g. the mean of $y$

2. At each iteration, a new tree $f_t$ is added to the model. The objective function at iteration $t$ is given by:

$$\mathcal{L}^{(t)} = \sum_{i = 1}^{n} l(y_i, \hat{y}_i^{(t-1)}) + f_{t}(x_i)) + \Omega(f_t)$$

3. For optimization, XGBoost uses first and second order Taylor expansion (gradient and Hessian) of the loss function to approximate the objective:

$$ \mathcal{L}^{(t)} \approx \sum_{i = 1}^{n} l(y_i, \hat{y}_i^{(t-1)}) + g_{i}f_{t}(x_i) + \frac{1}{2}h_{i}f_{t}^{2}(x_i)) + \Omega(f_t)$$

$$g_i = \frac{\partial l(y_i, \hat{y}_i)}{\partial \hat{y}_i} , h_i = \frac{\partial^2 l(y_i, \hat{y}_i)}{\partial \hat{y}_i^2}$$

This approximation allows XGBoost to apply a form of gradient-based optimization but specifically tailored to the structure of decision trees.

In our case, the loss function $l(y_i, \hat{y}_i)$ is $1/2$ times the *Sum of Squared Residuals* (SSR) as we are using regression. The $1/2$ term is included for the sake of mathematical convenience when computing the gradient and hessian. For the classification part of the project (done in `R`), the loss function would be log loss if binary and cross-entropy for multi-class. 

$$l(y_i, \hat{y}_i) = \frac{1}{2}(y_i - \hat{y}_i)^2$$

which means:

$g_i = \hat{y}_i - y_i$ and $h_i = 1$

Thus our loss objective simplifies to:

$$  \mathcal{L}^{(t)} \approx \sum_{i = 1}^{n} l(y_i, \hat{y}_i^{(t-1)}) + (\hat{y}_i^{(t-1)} - y_i)f_{t}(x_i) + \frac{1}{2}f_{t}^{2}(x_i)) + \Omega(f_t)$$

Since $l(y_i, \hat{y}_i^{(t-1)})$ does not depend on $f_t(x_i)$, it can be dropped from the optimization process. Thus the loss objective can then be written completely as:

$$\tilde{\mathcal{L}}^{(t)} \approx \sum_{i = 1}^{n} \left[ (\hat{y}_i^{(t-1)} - y_i)f_{t}(x_i) + \frac{1}{2}f_{t}^{2}(x_i) \right] + \gamma T + \frac{1}{2}\lambda \sum_{j = 1}^{T}w_j^2$$

$(\hat{y}_i^{(t-1)} - y_i)$ represents the residuals from the previous prediction and $f_t(x_i)$ is the new tree's prediction. 

To summarize, for tree constructions, we want to optimize this objective function. $g_i$ is the first-order derivative of the loss function and $h_i$ is the second order derivative which are both derived from Taylor expansion (not really in this case because of the simple loss function for regression). 

### Tree Construction

For tree building purposes, we will ignore the terminal node regularization term $\gamma T$ for now because pruning takes place *after* the full tree is built and plays no role in deriving the **Optimal Weight** $w^{*}_j$ for leaf $j$ (the optimal output at the one of the final leafs). Therefore the goal is to minimize $\tilde{\mathcal{L}}$

$$Gain = \frac{1}{2} \left[ \right]2$$

Long story short, *XGBoost* is an enhanced and more robust version of gradient boosting as described in ISLP. The two largest differences are regularization and optimization. In regards to regularization, *XGBoost* uses a more sophisticated and involved regularization process that allows for more involved tuning hyperparameters. For optimization, *XGBoost* uses first and second-order gradients (Hessians) as compared 